In [5]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
import os

# Dossier contenant les fichiers CSV
data_folder = 'data'

# Initialiser une liste pour stocker les DataFrames
dfs = []

# Boucler sur tous les fichiers CSV dans le dossier
for file in os.listdir(data_folder):
    if file.endswith('.csv'):
        # Construire le chemin complet du fichier
        file_path = os.path.join(data_folder, file)
        # Charger le fichier CSV dans un DataFrame
        df = pd.read_csv(file_path)
        # Ajouter le DataFrame à la liste
        dfs.append(df)

# Combiner tous les DataFrames en un seul
combined_df = pd.concat(dfs, ignore_index=True)

# Convertir les dates en datetime et calculer le chiffre d'affaires
combined_df['Order Date'] = pd.to_datetime(combined_df['Order Date'])
combined_df['Revenue'] = combined_df['Quantity'] * combined_df['Product Price']

# Agréger les données par semaine
weekly_revenue = combined_df.groupby(pd.Grouper(key='Order Date', freq='W'))['Revenue'].sum().reset_index()

In [6]:
# Préparer les données pour Prophet
df_for_prophet = weekly_revenue.rename(columns={'Order Date': 'ds', 'Revenue': 'y'}).dropna()

# Initialiser et entraîner le modèle Prophet
model = Prophet(weekly_seasonality=True, daily_seasonality=False)
model.fit(df_for_prophet)

# Faire des prédictions pour les 7 prochains jours
future = model.make_future_dataframe(periods=7)
forecast = model.predict(future)

# Visualiser les prévisions
model.plot(forecast)
plt.show()

17:33:39 - cmdstanpy - INFO - Chain [1] start processing
17:33:39 - cmdstanpy - INFO - Chain [1] done processing
17:33:39 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225653 
Optimization terminated abnormally. Falling back to Newton.
17:33:39 - cmdstanpy - INFO - Chain [1] start processing
17:33:39 - cmdstanpy - INFO - Chain [1] done processing
17:33:39 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225653 


RuntimeError: Error during optimization! Command 'C:\Logiciels\anaconda3\Lib\site-packages\prophet\stan_model\prophet_model.bin random seed=72574 data file=C:\Users\146657\AppData\Local\Temp\tmp8frkr000\qpt6wpd2.json init=C:\Users\146657\AppData\Local\Temp\tmp8frkr000\urydphkz.json output file=C:\Users\146657\AppData\Local\Temp\tmp8frkr000\prophet_model92h34cm2\prophet_model-20240925173339.csv method=optimize algorithm=newton iter=10000' failed: 

In [3]:
# Comparer les prévisions avec les données réelles (si disponibles)
from sklearn.metrics import mean_squared_error
import numpy as np

# Utiliser le DataFrame correct pour obtenir les vraies valeurs
y_true = df_for_prophet['y'].values[-7:]  # Les dernières 7 valeurs de l'ensemble d'entraînement
y_pred = forecast['yhat'].values[-7:]  # Les dernières 7 valeurs prédites

# Calculer le RMSE
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print(f"RMSE: {rmse}")

NameError: name 'forecast' is not defined